In [25]:
import pandas as pd
import json
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [2]:
with open ("time_unit_meme.json", "r") as f:
    posts_data = json.load(f)

In [3]:
with open("meme_table_combined_text.csv", "r") as f:
    meme_table = pd.read_csv(f)
meme_table.head()

,meme_id,cluster,created_utc,title,selftext,author,num_comments,score,extracted_text,comments_text,created_date,combined_text
0,cawlo8,cluster_4,1.562651e+09,"I have an MIDI keyboard, just no plug for it ):",NaN,YTMusicXD,0,3,When you want to easily make midi; limited pow...,NaN,2019-07-09,"i have an midi keyboard, just no plug for it )..."
1,d72m1n,cluster_4,1.569021e+09,R.I.P,NaN,masnargom,0,95,When Imina room full of antivax children and f...,NaN,2019-09-20,"r.i.p, when imina room full of antivax childre..."
2,bo6k2s,cluster_4,1.557769e+09,It's like a life hack,NaN,emanuelegay,2,4,Connect one Power Bank to another one Unlimite...,This is just plug in a power strip to itself\n...,2019-05-13,"it's like a life hack, connect one power bank ..."
3,cr40bc,cluster_4,1.565948e+09,I Finally Have Unlimited Power,NaN,Zaxoosh,0,64,When you finally get enough karma to post on r...,NaN,2019-08-16,"i finally have unlimited power, when you final..."
4,apoel0,cluster_4,1.549939e+09,Crrevt,NaN,memeboiiii7059,0,7,whwn yoy make a meme wirh one hand Unlimited p...,NaN,2019-02-12,"crrevt, whwn yoy make a meme wirh one hand unl..."


In [4]:
all_posts_df = pd.DataFrame(list(posts_data.items()), columns=['month', 'total_posts'])
all_posts_df['month'] = all_posts_df['month'].astype(int)

In [5]:
meme_table['month'] = pd.to_datetime(meme_table['created_date']).dt.to_period('M').apply(lambda x: x - pd.Period('2008-12', 'M')).apply(lambda x: x.n)

In [6]:
# Count occurrences of each cluster per month
cluster_month_counts = meme_table.groupby(['cluster', 'month']).size().reset_index(name='cluster_count')

In [7]:
# Merge with total posts per month
relative_frequency_df = cluster_month_counts.merge(all_posts_df, on='month', how='left')

In [8]:
# Calculate relative frequency
relative_frequency_df['relative_frequency'] = relative_frequency_df['cluster_count'] / relative_frequency_df['total_posts']

# Replace NaN with 0 for months with no posts
relative_frequency_df['relative_frequency'] = relative_frequency_df['relative_frequency'].fillna(0)

In [9]:
df = relative_frequency_df[['cluster', 'month', 'relative_frequency']]

In [10]:
df.head()

,cluster,month,relative_frequency
0,Acluster_0,120,0.000326
1,Acluster_0,123,0.000303
2,Acluster_0,124,0.000236
3,Acluster_0,125,0.000339
4,Acluster_0,126,0.000204


In [71]:
# content diversity in each time interval for cluster 81
diversity_81 = pd.read_csv("cluster_81_diversity.csv")
diversity_81.head()

,Month,Content Diversity
0,2018-02,0.000000
1,2018-03,0.708929
2,2018-04,0.625534
3,2018-05,0.620715
4,2018-06,0.837571


In [72]:
# Convert Month column to datetime
diversity_81['Month'] = pd.to_datetime(diversity_81['Month'], format='%Y-%m')

# Filter rows from 2019-01 to 2023-06
start_date = '2019-01-01'
end_date = '2023-12-30'  # End of June 2023
diversity_81 = diversity_81[(diversity_81['Month'] >= start_date) & (diversity_81['Month'] <= end_date)]

# Display the filtered DataFrame
diversity_81.head()

,Month,Content Diversity
9,2019-01-01,0.516535
10,2019-02-01,0.506728
11,2019-03-01,0.498619
12,2019-04-01,0.516828
13,2019-05-01,0.515982


In [73]:
cluster = df[df['cluster'] == "cluster_81"]
diversity_81['Month'] = pd.to_datetime(diversity_81['Month'])

# Calculate the month number (0 for 2008-12, 181 for 2023-12)
base_month = pd.Period('2008-12', freq='M')
diversity_81['Month Number'] = diversity_81['Month'].dt.to_period('M').apply(lambda x: x - base_month).apply(lambda x: x.n)

new_df = cluster.merge(diversity_81, left_on='month', right_on='Month Number', how='right')
new_df.head()

,cluster,month,relative_frequency,Month,Content Diversity,Month Number
0,cluster_81,121,0.000644,2019-01-01,0.516535,121
1,cluster_81,122,0.001547,2019-02-01,0.506728,122
2,cluster_81,123,0.000706,2019-03-01,0.498619,123
3,cluster_81,124,0.000471,2019-04-01,0.516828,124
4,cluster_81,125,0.000509,2019-05-01,0.515982,125


In [74]:
model = SARIMAX(new_df['relative_frequency'], exog=new_df[['Content Diversity']], order=(1, 0, 0))
result = model.fit()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -6.77252D+00    |proj g|=  1.56168D+02

At iterate    5    f= -6.77257D+00    |proj g|=  6.69323D+01

At iterate   10    f= -6.77257D+00    |proj g|=  2.72749D+01

At iterate   15    f= -6.77264D+00    |proj g|=  7.68772D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     15     76      1     0     0   7.688D+00  -6.773D+00
  F =  -6.7726396990143876     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


 This problem is unconstrained.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


In [75]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:     relative_frequency   No. Observations:                   60
Model:               SARIMAX(1, 0, 0)   Log Likelihood                 406.358
Date:                Wed, 11 Dec 2024   AIC                           -806.717
Time:                        10:06:32   BIC                           -800.434
Sample:                             0   HQIC                          -804.259
                                 - 60                                         
Covariance Type:                  opg                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Content Diversity     0.0016      0.000     12.881      0.000       0.001       0.002
ar.L1                 0.3659      0.137      2.663      0.008       0.097       0.635
sigma2             7.395e-08   1.61e-08      4.582      0.000    4.23e-08    1.06e-07
===================================================================================
Ljung-Box (L1) (Q):                   0.29   Jarque-Bera (JB):                 4.53
Prob(Q):                              0.59   Prob(JB):                         0.10
Heteroskedasticity (H):               1.65   Skew:                             0.67
Prob(H) (two-sided):                  0.27   Kurtosis:                         2.96
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [76]:
model = SARIMAX(new_df['Content Diversity'], exog=new_df[['relative_frequency']], order=(1, 0, 0))
result = model.fit()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -4.23378D-01    |proj g|=  1.72143D-02

At iterate    5    f= -4.23382D-01    |proj g|=  7.79091D-03

At iterate   10    f= -4.23521D-01    |proj g|=  9.75470D-02

At iterate   15    f= -4.36493D-01    |proj g|=  1.01731D+00

At iterate   20    f= -1.60977D+00    |proj g|=  7.47968D+00

At iterate   25    f= -1.67932D+00    |proj g|=  7.01951D-01

At iterate   30    f= -1.68172D+00    |proj g|=  8.86918D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nac

 This problem is unconstrained.


In [77]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:      Content Diversity   No. Observations:                   60
Model:               SARIMAX(1, 0, 0)   Log Likelihood                 100.903
Date:                Wed, 11 Dec 2024   AIC                           -195.807
Time:                        10:06:53   BIC                           -189.524
Sample:                             0   HQIC                          -193.349
                                 - 60                                         
Covariance Type:                  opg                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
relative_frequency   -17.7347     15.730     -1.127      0.260     -48.565      13.096
ar.L1                  0.9960      0.011     87.597      0.000       0.974       1.018
sigma2                 0.0019      0.000      8.081      0.000       0.001       0.002
===================================================================================
Ljung-Box (L1) (Q):                   6.78   Jarque-Bera (JB):                57.19
Prob(Q):                              0.01   Prob(JB):                         0.00
Heteroskedasticity (H):               9.63   Skew:                            -1.27
Prob(H) (two-sided):                  0.00   Kurtosis:                         7.05
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [78]:
results_12 = grangercausalitytests(new_df[['relative_frequency', 'Content Diversity']], maxlag=4)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0160  , p=0.8998  , df_denom=56, df_num=1
ssr based chi2 test:   chi2=0.0169  , p=0.8967  , df=1
likelihood ratio test: chi2=0.0169  , p=0.8967  , df=1
parameter F test:         F=0.0160  , p=0.8998  , df_denom=56, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.0121  , p=0.9879  , df_denom=53, df_num=2
ssr based chi2 test:   chi2=0.0266  , p=0.9868  , df=2
likelihood ratio test: chi2=0.0266  , p=0.9868  , df=2
parameter F test:         F=0.0121  , p=0.9879  , df_denom=53, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.9984  , p=0.4014  , df_denom=50, df_num=3
ssr based chi2 test:   chi2=3.4144  , p=0.3320  , df=3
likelihood ratio test: chi2=3.3161  , p=0.3454  , df=3
parameter F test:         F=0.9984  , p=0.4014  , df_denom=50, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=2.7252  , p=0.0403  , df_d

In [79]:
results_21 = grangercausalitytests(new_df[['Content Diversity', 'relative_frequency']], maxlag=4)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=5.3796  , p=0.0240  , df_denom=56, df_num=1
ssr based chi2 test:   chi2=5.6678  , p=0.0173  , df=1
likelihood ratio test: chi2=5.4119  , p=0.0200  , df=1
parameter F test:         F=5.3796  , p=0.0240  , df_denom=56, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.6295  , p=0.0815  , df_denom=53, df_num=2
ssr based chi2 test:   chi2=5.7551  , p=0.0563  , df=2
likelihood ratio test: chi2=5.4872  , p=0.0643  , df=2
parameter F test:         F=2.6295  , p=0.0815  , df_denom=53, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=2.2129  , p=0.0981  , df_denom=50, df_num=3
ssr based chi2 test:   chi2=7.5680  , p=0.0558  , df=3
likelihood ratio test: chi2=7.1061  , p=0.0686  , df=3
parameter F test:         F=2.2129  , p=0.0981  , df_denom=50, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=1.7037  , p=0.1650  , df_d

In [55]:
# content diversity in each time interval for cluster 175
name = "cluster_175"
diversity_one = pd.read_csv(f"{name}_diversity.csv")
diversity_one.head(20)

,Month,Content Diversity
0,2018-02,0.000000
1,2018-04,0.755550
2,2018-05,0.753675
3,2018-06,0.782215
4,2018-08,0.809006
5,2018-09,0.658521
6,2018-10,0.689833
7,2018-11,0.664557
8,2018-12,0.647292
9,2019-01,0.630443


In [56]:
# Convert Month column to datetime
diversity_one['Month'] = pd.to_datetime(diversity_one['Month'], format='%Y-%m')

# Filter rows from 2019-01 to 2023-06
start_date = '2019-01-01'
end_date = '2023-06-30'  # End of June 2023
filtered = diversity_one[(diversity_one['Month'] >= start_date) & (diversity_one['Month'] <= end_date)]

# Display the filtered DataFrame
filtered.head()

,Month,Content Diversity
9,2019-01-01,0.630443
10,2019-02-01,0.587148
11,2019-03-01,0.583210
12,2019-04-01,0.630210
13,2019-05-01,0.605135


In [57]:
len(filtered)

54

In [59]:
cluster = df[df['cluster'] == name]
filtered['Month'] = pd.to_datetime(filtered['Month'])

# Calculate the month number (0 for 2008-12, 181 for 2023-12)
base_month = pd.Period('2008-12', freq='M')
filtered['Month Number'] = filtered['Month'].dt.to_period('M').apply(lambda x: x - base_month).apply(lambda x: x.n)

new_df = cluster.merge(filtered, left_on='month', right_on='Month Number', how='right')
new_df.head()

/var/folders/rg/p_ps2y216432jwhfjvk8nh8m0000gn/T/ipykernel_68201/2442282076.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['Month'] = pd.to_datetime(filtered['Month'])
/var/folders/rg/p_ps2y216432jwhfjvk8nh8m0000gn/T/ipykernel_68201/2442282076.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['Month Number'] = filtered['Month'].dt.to_period('M').apply(lambda x: x - base_month).apply(lambda x: x.n)


,cluster,month,relative_frequency,Month,Content Diversity,Month Number
0,cluster_175,121,0.002361,2019-01-01,0.630443,121
1,cluster_175,122,0.001701,2019-02-01,0.587148,122
2,cluster_175,123,0.001109,2019-03-01,0.583210,123
3,cluster_175,124,0.000864,2019-04-01,0.630210,124
4,cluster_175,125,0.001356,2019-05-01,0.605135,125


In [60]:
model = SARIMAX(new_df['relative_frequency'], exog=new_df[['Content Diversity']], order=(1, 0, 0))
result = model.fit()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -6.41682D+00    |proj g|=  8.39922D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3      2     47      2     0     0   5.271D+01  -6.418D+00
  F =  -6.4176622799261454     

ABNORMAL_TERMINATION_IN_LNSRCH                              


 This problem is unconstrained.

 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.
/Users/yingrongmao/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.


In [61]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:     relative_frequency   No. Observations:                   54
Model:               SARIMAX(1, 0, 0)   Log Likelihood                 346.554
Date:                Wed, 11 Dec 2024   AIC                           -687.108
Time:                        10:00:33   BIC                           -681.141
Sample:                             0   HQIC                          -684.806
                                 - 54                                         
Covariance Type:                  opg                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Content Diversity     0.0024      0.000     17.855      0.000       0.002       0.003
ar.L1                 0.3694      0.151      2.447      0.014       0.074       0.665
sigma2              1.55e-07   3.26e-08      4.760      0.000    9.12e-08    2.19e-07
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):                 0.78
Prob(Q):                              0.91   Prob(JB):                         0.68
Heteroskedasticity (H):               1.51   Skew:                            -0.26
Prob(H) (two-sided):                  0.39   Kurtosis:                         2.73
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [62]:
results_12 = grangercausalitytests(new_df[['relative_frequency', 'Content Diversity']], maxlag=4)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0316  , p=0.8597  , df_denom=50, df_num=1
ssr based chi2 test:   chi2=0.0335  , p=0.8549  , df=1
likelihood ratio test: chi2=0.0334  , p=0.8549  , df=1
parameter F test:         F=0.0316  , p=0.8597  , df_denom=50, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.3001  , p=0.7422  , df_denom=47, df_num=2
ssr based chi2 test:   chi2=0.6640  , p=0.7175  , df=2
likelihood ratio test: chi2=0.6598  , p=0.7190  , df=2
parameter F test:         F=0.3001  , p=0.7422  , df_denom=47, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.9048  , p=0.1427  , df_denom=44, df_num=3
ssr based chi2 test:   chi2=6.6236  , p=0.0849  , df=3
likelihood ratio test: chi2=6.2275  , p=0.1011  , df=3
parameter F test:         F=1.9048  , p=0.1427  , df_denom=44, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=1.3884  , p=0.2549  , df_d

In [63]:
results_21 = grangercausalitytests(new_df[['Content Diversity', 'relative_frequency']], maxlag=4)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.8956  , p=0.3485  , df_denom=50, df_num=1
ssr based chi2 test:   chi2=0.9494  , p=0.3299  , df=1
likelihood ratio test: chi2=0.9410  , p=0.3320  , df=1
parameter F test:         F=0.8956  , p=0.3485  , df_denom=50, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.1136  , p=0.3369  , df_denom=47, df_num=2
ssr based chi2 test:   chi2=2.4642  , p=0.2917  , df=2
likelihood ratio test: chi2=2.4076  , p=0.3001  , df=2
parameter F test:         F=1.1136  , p=0.3369  , df_denom=47, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.6317  , p=0.1957  , df_denom=44, df_num=3
ssr based chi2 test:   chi2=5.6740  , p=0.1286  , df=3
likelihood ratio test: chi2=5.3800  , p=0.1460  , df=3
parameter F test:         F=1.6317  , p=0.1957  , df_denom=44, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=1.5157  , p=0.2155  , df_d

In [32]:
# content diversity in each time interval for cluster 175
name = "cluster_5"
diversity_one = pd.read_csv(f"{name}_diversity.csv")
diversity_one.head()

,Month,Content Diversity
0,2018-11,0.000000
1,2018-12,0.665364
2,2019-01,0.580256
3,2019-02,0.791888
4,2019-03,0.646392


In [64]:
# Convert Month column to datetime
diversity_one['Month'] = pd.to_datetime(diversity_one['Month'], format='%Y-%m')

# Filter rows from 2019-01 to 2023-06
start_date = '2019-01-01'
end_date = '2022-01-01'  # End of June 2023
diversity_one = diversity_one[(diversity_one['Month'] >= start_date) & (diversity_one['Month'] <= end_date)]

# Display the filtered DataFrame
diversity_one.head()

,Month,Content Diversity
9,2019-01-01,0.630443
10,2019-02-01,0.587148
11,2019-03-01,0.583210
12,2019-04-01,0.630210
13,2019-05-01,0.605135


In [66]:
cluster = df[df['cluster'] == name]
diversity_one['Month'] = pd.to_datetime(diversity_one['Month'])

# Calculate the month number (0 for 2008-12, 181 for 2023-12)
base_month = pd.Period('2008-12', freq='M')
diversity_one['Month Number'] = diversity_one['Month'].dt.to_period('M').apply(lambda x: x - base_month).apply(lambda x: x.n)

new_df = cluster.merge(diversity_one, left_on='month', right_on='Month Number', how='right')
new_df.head()

,cluster,month,relative_frequency,Month,Content Diversity,Month Number
0,cluster_175,121,0.002361,2019-01-01,0.630443,121
1,cluster_175,122,0.001701,2019-02-01,0.587148,122
2,cluster_175,123,0.001109,2019-03-01,0.583210,123
3,cluster_175,124,0.000864,2019-04-01,0.630210,124
4,cluster_175,125,0.001356,2019-05-01,0.605135,125


In [67]:
model = SARIMAX(new_df['relative_frequency'], exog=new_df[['Content Diversity']], order=(1, 0, 0))
result = model.fit()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -6.52094D+00    |proj g|=  1.45566D+02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3      2     20      1     0     0   7.499D+01  -6.523D+00
  F =  -6.5225167023678496     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


 This problem is unconstrained.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


In [68]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:     relative_frequency   No. Observations:                   37
Model:               SARIMAX(1, 0, 0)   Log Likelihood                 241.333
Date:                Wed, 11 Dec 2024   AIC                           -476.666
Time:                        10:03:39   BIC                           -471.833
Sample:                             0   HQIC                          -474.962
                                 - 37                                         
Covariance Type:                  opg                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Content Diversity     0.0026      0.000     17.961      0.000       0.002       0.003
ar.L1                 0.3824      0.201      1.901      0.057      -0.012       0.777
sigma2             1.259e-07   3.28e-08      3.836      0.000    6.16e-08     1.9e-07
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):                 0.11
Prob(Q):                              0.96   Prob(JB):                         0.95
Heteroskedasticity (H):               0.61   Skew:                            -0.13
Prob(H) (two-sided):                  0.40   Kurtosis:                         2.99
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [69]:
results_21 = grangercausalitytests(new_df[['Content Diversity', 'relative_frequency']], maxlag=4)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.1800  , p=0.2852  , df_denom=33, df_num=1
ssr based chi2 test:   chi2=1.2873  , p=0.2565  , df=1
likelihood ratio test: chi2=1.2648  , p=0.2607  , df=1
parameter F test:         F=1.1800  , p=0.2852  , df_denom=33, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.1162  , p=0.1381  , df_denom=30, df_num=2
ssr based chi2 test:   chi2=4.9378  , p=0.0847  , df=2
likelihood ratio test: chi2=4.6192  , p=0.0993  , df=2
parameter F test:         F=2.1162  , p=0.1381  , df_denom=30, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.3759  , p=0.2713  , df_denom=27, df_num=3
ssr based chi2 test:   chi2=5.1978  , p=0.1579  , df=3
likelihood ratio test: chi2=4.8368  , p=0.1841  , df=3
parameter F test:         F=1.3759  , p=0.2713  , df_denom=27, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=1.7383  , p=0.1745  , df_d

In [70]:
results_12 = grangercausalitytests(new_df[['relative_frequency', 'Content Diversity']], maxlag=4)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.8196  , p=0.3719  , df_denom=33, df_num=1
ssr based chi2 test:   chi2=0.8941  , p=0.3444  , df=1
likelihood ratio test: chi2=0.8832  , p=0.3473  , df=1
parameter F test:         F=0.8196  , p=0.3719  , df_denom=33, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.6261  , p=0.5415  , df_denom=30, df_num=2
ssr based chi2 test:   chi2=1.4610  , p=0.4817  , df=2
likelihood ratio test: chi2=1.4313  , p=0.4889  , df=2
parameter F test:         F=0.6261  , p=0.5415  , df_denom=30, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.6638  , p=0.5816  , df_denom=27, df_num=3
ssr based chi2 test:   chi2=2.5076  , p=0.4739  , df=3
likelihood ratio test: chi2=2.4195  , p=0.4900  , df=3
parameter F test:         F=0.6638  , p=0.5816  , df_denom=27, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.8735  , p=0.4942  , df_d

In [91]:
name = "cluster_81"
diversity_one = pd.read_csv(f"{name}_diversity.csv")
diversity_one[diversity_one["Month"]=="2018-12"]

,Month,Content Diversity
8,2018-12,0.5346


In [89]:
name = "cluster_81"
diversity_one = pd.read_csv(f"{name}_diversity.csv")
diversity_one.tail()    

,Month,Content Diversity
64,2023-08,0.586380
65,2023-09,0.646191
66,2023-10,0.563774
67,2023-11,0.564521
68,2023-12,0.385074


In [41]:
name = "Acluster_290"
diversity_one = pd.read_csv(f"{name}_diversity.csv")

cluster = df[df['cluster'] == name]
diversity_one['Month'] = pd.to_datetime(diversity_one['Month'])

# Calculate the month number (0 for 2008-12, 181 for 2023-12)
base_month = pd.Period('2008-12', freq='M')
diversity_one['Month Number'] = diversity_one['Month'].dt.to_period('M').apply(lambda x: x - base_month).apply(lambda x: x.n)

new_df = cluster.merge(diversity_one, left_on='month', right_on='Month Number', how='left')

model = SARIMAX(new_df['relative_frequency'], exog=new_df[['Content Diversity']], order=(1, 0, 0))
result = model.fit()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -6.54247D+00    |proj g|=  2.59826D+02

At iterate    5    f= -6.54587D+00    |proj g|=  4.84669D+01

At iterate   10    f= -6.54588D+00    |proj g|=  1.73356D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     15     85      2     0     0   7.438D-01  -6.546D+00
  F =  -6.5459047546623594     

ABNORMAL_TERMINATION_IN_LNSRCH                              


 This problem is unconstrained.

 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.
/Users/yingrongmao/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.


In [42]:
print(result.summary())

print(grangercausalitytests(new_df[['Content Diversity', 'relative_frequency']], maxlag=4))

print(grangercausalitytests(new_df[['relative_frequency', 'Content Diversity']], maxlag=4))

                               SARIMAX Results                                
Dep. Variable:     relative_frequency   No. Observations:                   58
Model:               SARIMAX(1, 0, 0)   Log Likelihood                 379.662
Date:                Wed, 11 Dec 2024   AIC                           -753.325
Time:                        00:04:04   BIC                           -747.144
Sample:                             0   HQIC                          -750.917
                                 - 58                                         
Covariance Type:                  opg                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Content Diversity     0.0013      0.000      9.442      0.000       0.001       0.002
ar.L1                 0.4017      0.091      4.423      0.000       0.224       0.580
sigma2             1.169

In [84]:
name = "cluster_34"
diversity_one = pd.read_csv(f"{name}_diversity.csv")
# Convert Month column to datetime
diversity_one['Month'] = pd.to_datetime(diversity_one['Month'], format='%Y-%m')

# Filter rows from 2019-01 to 2023-06
start_date = '2019-05-01'
end_date = '2022-06-30'  # End of June 2023
diversity_one = diversity_one[(diversity_one['Month'] >= start_date) & (diversity_one['Month'] <= end_date)]

# Display the filtered DataFrame
filtered.head()
cluster = df[df['cluster'] == name]

# Calculate the month number (0 for 2008-12, 181 for 2023-12)
base_month = pd.Period('2008-12', freq='M')
diversity_one['Month Number'] = diversity_one['Month'].dt.to_period('M').apply(lambda x: x - base_month).apply(lambda x: x.n)

new_df = cluster.merge(diversity_one, left_on='month', right_on='Month Number', how='right')

model = SARIMAX(new_df['relative_frequency'], exog=new_df[['Content Diversity']], order=(1, 0, 0))
result = model.fit()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -6.65798D+00    |proj g|=  3.72647D+03

At iterate    5    f= -6.77717D+00    |proj g|=  7.27639D+01

At iterate   10    f= -6.77723D+00    |proj g|=  7.76724D+01

At iterate   15    f= -6.77796D+00    |proj g|=  2.08743D-01

At iterate   20    f= -6.77797D+00    |proj g|=  3.50740D+00

At iterate   25    f= -6.77831D+00    |proj g|=  3.80155D+01

At iterate   30    f= -6.80380D+00    |proj g|=  2.31117D+02

At iterate   35    f= -6.82878D+00    |proj g|=  5.87429D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function 

 This problem is unconstrained.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


In [85]:
print(result.summary())

print(grangercausalitytests(new_df[['Content Diversity', 'relative_frequency']], maxlag=4))

print(grangercausalitytests(new_df[['relative_frequency', 'Content Diversity']], maxlag=4))

                               SARIMAX Results                                
Dep. Variable:     relative_frequency   No. Observations:                   38
Model:               SARIMAX(1, 0, 0)   Log Likelihood                 259.498
Date:                Wed, 11 Dec 2024   AIC                           -512.997
Time:                        12:09:54   BIC                           -508.084
Sample:                             0   HQIC                          -511.249
                                 - 38                                         
Covariance Type:                  opg                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Content Diversity     0.0010      0.000      2.485      0.013       0.000       0.002
ar.L1                 0.8131      0.073     11.163      0.000       0.670       0.956
sigma2             6.414

In [45]:
name = "cluster_200"
diversity_one = pd.read_csv(f"{name}_diversity.csv")

cluster = df[df['cluster'] == name]
diversity_one['Month'] = pd.to_datetime(diversity_one['Month'])

# Calculate the month number (0 for 2008-12, 181 for 2023-12)
base_month = pd.Period('2008-12', freq='M')
diversity_one['Month Number'] = diversity_one['Month'].dt.to_period('M').apply(lambda x: x - base_month).apply(lambda x: x.n)

new_df = cluster.merge(diversity_one, left_on='month', right_on='Month Number', how='left')

model = SARIMAX(new_df['relative_frequency'], exog=new_df[['Content Diversity']], order=(1, 0, 0))
result = model.fit()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -6.58171D+00    |proj g|=  1.24767D+02

At iterate    5    f= -6.58543D+00    |proj g|=  5.12888D-01

At iterate   10    f= -6.58543D+00    |proj g|=  1.22832D-01

At iterate   15    f= -6.58544D+00    |proj g|=  1.69849D+00

At iterate   20    f= -6.58555D+00    |proj g|=  1.72416D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     24     44      1     0     0   2.919D+01  -6.586D+00
  F =  -6.5857357521851423     

CONVERG

 This problem is unconstrained.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


In [46]:
print(result.summary())

print(grangercausalitytests(new_df[['Content Diversity', 'relative_frequency']], maxlag=4))

print(grangercausalitytests(new_df[['relative_frequency', 'Content Diversity']], maxlag=4))

                               SARIMAX Results                                
Dep. Variable:     relative_frequency   No. Observations:                   58
Model:               SARIMAX(1, 0, 0)   Log Likelihood                 381.973
Date:                Wed, 11 Dec 2024   AIC                           -757.945
Time:                        00:12:04   BIC                           -751.764
Sample:                             0   HQIC                          -755.538
                                 - 58                                         
Covariance Type:                  opg                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Content Diversity     0.0009      0.000      6.906      0.000       0.001       0.001
ar.L1                 0.4799      0.118      4.083      0.000       0.250       0.710
sigma2             1.088

In [82]:
name = "cluster_318"
diversity_one = pd.read_csv(f"{name}_diversity.csv")

diversity_one['Month'] = pd.to_datetime(diversity_one['Month'], format='%Y-%m')

# Filter rows from 2019-01 to 2023-06
start_date = '2019-07-01'
end_date = '2022-11-30'  # End of June 2023
diversity_one = diversity_one[(diversity_one['Month'] >= start_date) & (diversity_one['Month'] <= end_date)]

# Display the filtered DataFrame
diversity_one.head()

cluster = df[df['cluster'] == name]
diversity_one['Month'] = pd.to_datetime(diversity_one['Month'])

# Calculate the month number (0 for 2008-12, 181 for 2023-12)
base_month = pd.Period('2008-12', freq='M')
diversity_one['Month Number'] = diversity_one['Month'].dt.to_period('M').apply(lambda x: x - base_month).apply(lambda x: x.n)

new_df = cluster.merge(diversity_one, left_on='month', right_on='Month Number', how='right')

model = SARIMAX(new_df['relative_frequency'], exog=new_df[['Content Diversity']], order=(1, 0, 0))
result = model.fit()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -6.79871D+00    |proj g|=  8.50201D+01

At iterate    5    f= -6.80360D+00    |proj g|=  2.84783D+02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3      7     34      1     0     0   2.669D+00  -6.807D+00
  F =  -6.8068257024272487     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


 This problem is unconstrained.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


In [83]:
print(result.summary())

print(grangercausalitytests(new_df[['Content Diversity', 'relative_frequency']], maxlag=4))

print(grangercausalitytests(new_df[['relative_frequency', 'Content Diversity']], maxlag=4))

                               SARIMAX Results                                
Dep. Variable:     relative_frequency   No. Observations:                   41
Model:               SARIMAX(1, 0, 0)   Log Likelihood                 279.080
Date:                Wed, 11 Dec 2024   AIC                           -552.160
Time:                        12:05:13   BIC                           -547.019
Sample:                             0   HQIC                          -550.288
                                 - 41                                         
Covariance Type:                  opg                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Content Diversity     0.0010      0.000      3.938      0.000       0.001       0.002
ar.L1                 0.6803      0.127      5.365      0.000       0.432       0.929
sigma2             6.798

In [49]:
name = "cluster_609"
diversity_one = pd.read_csv(f"{name}_diversity.csv")

cluster = df[df['cluster'] == name]
diversity_one['Month'] = pd.to_datetime(diversity_one['Month'])

# Calculate the month number (0 for 2008-12, 181 for 2023-12)
base_month = pd.Period('2008-12', freq='M')
diversity_one['Month Number'] = diversity_one['Month'].dt.to_period('M').apply(lambda x: x - base_month).apply(lambda x: x.n)

new_df = cluster.merge(diversity_one, left_on='month', right_on='Month Number', how='left')

model = SARIMAX(new_df['relative_frequency'], exog=new_df[['Content Diversity']], order=(1, 0, 0))
result = model.fit()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -6.86380D+00    |proj g|=  8.08427D+02

At iterate    5    f= -6.87613D+00    |proj g|=  1.40498D+02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3      8     65      1     0     0   1.363D+02  -6.876D+00
  F =  -6.8761299983157862     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


 This problem is unconstrained.


In [50]:
print(result.summary())

print(grangercausalitytests(new_df[['Content Diversity', 'relative_frequency']], maxlag=4))

print(grangercausalitytests(new_df[['relative_frequency', 'Content Diversity']], maxlag=4))

                               SARIMAX Results                                
Dep. Variable:     relative_frequency   No. Observations:                   61
Model:               SARIMAX(1, 0, 0)   Log Likelihood                 419.444
Date:                Wed, 11 Dec 2024   AIC                           -832.888
Time:                        00:12:08   BIC                           -826.555
Sample:                             0   HQIC                          -830.406
                                 - 61                                         
Covariance Type:                  opg                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Content Diversity     0.0008   5.93e-05     13.536      0.000       0.001       0.001
ar.L1                 0.0851      0.112      0.760      0.447      -0.134       0.304
sigma2             6.213

In [51]:
name = "cluster_694"
diversity_one = pd.read_csv(f"{name}_diversity.csv")

cluster = df[df['cluster'] == name]
diversity_one['Month'] = pd.to_datetime(diversity_one['Month'])

# Calculate the month number (0 for 2008-12, 181 for 2023-12)
base_month = pd.Period('2008-12', freq='M')
diversity_one['Month Number'] = diversity_one['Month'].dt.to_period('M').apply(lambda x: x - base_month).apply(lambda x: x.n)

new_df = cluster.merge(diversity_one, left_on='month', right_on='Month Number', how='left')

model = SARIMAX(new_df['relative_frequency'], exog=new_df[['Content Diversity']], order=(1, 0, 0))
result = model.fit()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -6.70413D+00    |proj g|=  1.34573D+02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3      5     53      2     0     0   5.729D+01  -6.712D+00
  F =  -6.7115934746528811     

ABNORMAL_TERMINATION_IN_LNSRCH                              


 This problem is unconstrained.

 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.
/Users/yingrongmao/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.


In [52]:
print(result.summary())

print(grangercausalitytests(new_df[['Content Diversity', 'relative_frequency']], maxlag=4))

print(grangercausalitytests(new_df[['relative_frequency', 'Content Diversity']], maxlag=4))

                               SARIMAX Results                                
Dep. Variable:     relative_frequency   No. Observations:                   63
Model:               SARIMAX(1, 0, 0)   Log Likelihood                 422.830
Date:                Wed, 11 Dec 2024   AIC                           -839.661
Time:                        00:12:09   BIC                           -833.231
Sample:                             0   HQIC                          -837.132
                                 - 63                                         
Covariance Type:                  opg                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Content Diversity     0.0010      0.000      9.252      0.000       0.001       0.001
ar.L1                 0.5046      0.119      4.236      0.000       0.271       0.738
sigma2             8.486

In [53]:
name = "cluster_833"
diversity_one = pd.read_csv(f"{name}_diversity.csv")

cluster = df[df['cluster'] == name]
diversity_one['Month'] = pd.to_datetime(diversity_one['Month'])

# Calculate the month number (0 for 2008-12, 181 for 2023-12)
base_month = pd.Period('2008-12', freq='M')
diversity_one['Month Number'] = diversity_one['Month'].dt.to_period('M').apply(lambda x: x - base_month).apply(lambda x: x.n)

new_df = cluster.merge(diversity_one, left_on='month', right_on='Month Number', how='left')

model = SARIMAX(new_df['relative_frequency'], exog=new_df[['Content Diversity']], order=(1, 0, 0))
result = model.fit()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -6.80515D+00    |proj g|=  2.85614D+02

At iterate    5    f= -6.81007D+00    |proj g|=  1.04014D+02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3      8     51      2     0     0   5.861D-02  -6.811D+00
  F =  -6.8108030732619778     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


 This problem is unconstrained.

 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


In [54]:
print(result.summary())

print(grangercausalitytests(new_df[['Content Diversity', 'relative_frequency']], maxlag=4))

print(grangercausalitytests(new_df[['relative_frequency', 'Content Diversity']], maxlag=4))

                               SARIMAX Results                                
Dep. Variable:     relative_frequency   No. Observations:                   63
Model:               SARIMAX(1, 0, 0)   Log Likelihood                 429.081
Date:                Wed, 11 Dec 2024   AIC                           -852.161
Time:                        00:12:11   BIC                           -845.732
Sample:                             0   HQIC                          -849.632
                                 - 63                                         
Covariance Type:                  opg                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Content Diversity     0.0013      0.000     11.303      0.000       0.001       0.001
ar.L1                 0.4023      0.117      3.452      0.001       0.174       0.631
sigma2             6.823